In [74]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# NLTK
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

## Data Loading 

In [3]:
df = pd.read_json('./JEOPARDY_QUESTIONS1.json')

In [4]:
df.head()

,category,air_date,question,value,answer,round,show_number
0,HISTORY,2004-12-31,"'For the last 8 years of his life, Galileo was...",$200,Copernicus,Jeopardy!,4680
1,ESPN's TOP 10 ALL-TIME ATHLETES,2004-12-31,'No. 2: 1912 Olympian; football star at Carlis...,$200,Jim Thorpe,Jeopardy!,4680
2,EVERYBODY TALKS ABOUT IT...,2004-12-31,'The city of Yuma in this state has a record a...,$200,Arizona,Jeopardy!,4680
3,THE COMPANY LINE,2004-12-31,"'In 1963, live on ""The Art Linkletter Show"", t...",$200,McDonald\'s,Jeopardy!,4680
4,EPITAPHS & TRIBUTES,2004-12-31,"'Signer of the Dec. of Indep., framer of the C...",$200,John Adams,Jeopardy!,4680


In [5]:
questions = df['question']

## Preprocessing

### Tokenization

In [6]:
def tokenization(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

tokenized_sentences = list(tokenization(questions))

print(tokenized_sentences [1:3])

[['no', 'olympian', 'football', 'star', 'at', 'carlisle', 'indian', 'school', 'mlb', 'seasons', 'with', 'the', 'reds', 'giants', 'braves'], ['the', 'city', 'of', 'yuma', 'in', 'this', 'state', 'has', 'record', 'average', 'of', 'hours', 'of', 'sunshine', 'each', 'year']]


In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(tokenized_sentences, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[tokenized_sentences], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[tokenized_sentences[0]]])

In [8]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [9]:
# Remove Stop Words
tokenized_without_stopwords = remove_stopwords(tokenized_sentences)

# Form Bigrams
tokenized_without_stopwords_bigrams = make_bigrams(tokenized_without_stopwords)


nlp = spacy.load("en_core_web_sm")

# Do lemmatization keeping only noun, adj, vb, adv
lemmatized_words = lemmatization(tokenized_without_stopwords_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


C:\Users\enala_culjkpz\AppData\Roaming\Python\Python38\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'en_core_web_sm' (3.1.0) was trained with spaCy v3.1 and may not be 100% compatible with the current version (3.3.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [73]:
print(lemmatized_words[1:9])

[['olympian', 'season', 'red', 'giant', 'brave'], ['city', 'state', 'record', 'average', 'hour', 'sunshine', 'year'], ['live', 'art', 'linkletter', 'show', 'company', 'serve', 'billionth', 'burger'], ['second', 'president', 'united_state'], ['title', 'aesop_fable', 'insect', 'share', 'billing', 'grasshopper'], ['build', 'still', 'use', 'today'], ['steal', 'baron', 'steal', 'bull'], ['winter', 'record', 'inch', 'snow', 'fall', 'ranger', 'station', 'state']]


In [11]:
# Create Dictionary
dictionary = corpora.Dictionary(lemmatized_words)

# Create Corpus
texts = lemmatized_words

# Term Document Frequency
corpus = [dictionary.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]]


In [80]:
# Create Dictionary
dictionary = corpora.Dictionary(lemmatized_words)

# Create Corpus
texts = lemmatized_words

# Term Document Frequency
corpus = [dictionary.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]]


In [33]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,  id2word=dictionary,num_topics= 6)

In [68]:
doc_lda = lda_model[corpus]
for index, topics in lda_model.show_topics(formatted=False, num_words=50):
        print('Topic: {} \nWords: {}'.format(index+1,[w[0] for w in topics] ))


Topic: 1 
Words: ['name', 'br', 'city', 'country', 'first', 'state', 'large', 'island', 'capital', 'world', 'word', 'mean', 'know', 'people', 'good', 'found', 'call', 'become', 'king', 'bear', 'old', 'group', 'year', 'find', 'come', 'give', 'person', 'new', 'letter', 'long', 'river', 'begin', 'part', 'take', 'sea', 'area', 'son', 'also', 'share', 'french', 'latin', 'land', 'last', 'build', 'lie', 'home', 'high', 'use', 'spanish', 'end']
Topic: 2 
Words: ['play', 'first', 'film', 'title', 'write', 'year', 'man', 'star', 'novel', 'book', 'become', 'woman', 'show', 'hit', 'time', 'character', 'say', 'work', 'tv', 'base', 'movie', 'last', 'author', 'war', 'see', 'day', 'story', 'life', 'musical', 'take', 'old', 'begin', 'music', 'bear', 'go', 'die', 'series', 'actor', 'member', 'tell', 'child', 'game', 'record', 'president', 'include', 'get', 'love', 'new', 'hold', 'world']
Topic: 3 
Words: ['song', 'name', 'state', 'make', 'mile', 'run', 'create', 'city', 'river', 'open', 'nation', 'count

In [84]:
report="This chilled leek & potato soup is traditionally topped with chopped chives"
report = report.split(" ")

for index, proportion in lda_model[dictionary.doc2bow(report)]:
    print('The proportion of topic {} is {} percent'.format(index+1, proportion*100))


The proportion of topic 1 is 3.2206781208515167
The proportion of topic 2 is 3.220515325665474
The proportion of topic 3 is 3.2211963087320328
The proportion of topic 4 is 3.220806270837784
The proportion of topic 5 is 83.8956892490387
The proportion of topic 6 is 3.221116214990616


In [76]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, dictionary, sort_topics = True)

C:\Users\enala_culjkpz\AppData\Roaming\Python\Python38\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.077279  0.259501       1        1  21.273304
0     -0.131466  0.041907       2        1  18.771740
3     -0.051129 -0.232004       3        1  17.459512
5      0.416593  0.017496       4        1  14.786469
4     -0.076932 -0.182440       5        1  13.953721
2     -0.079787  0.095539       6        1  13.755254, topic_info=        Term          Freq         Total Category  logprob  loglift
153      com  13944.000000  13944.000000  Default  30.0000  30.0000
158   medium  13914.000000  13914.000000  Default  29.0000  29.0000
150  archive  13812.000000  13812.000000  Default  28.0000  28.0000
161      www  13810.000000  13810.000000  Default  27.0000  27.0000
157     http  13543.000000  13543.000000  Default  26.0000  26.0000
..       ...           ...           ...      ...      ...      ...
107  country   1119.275189   5470.775743   Topic6  -5.1125   0.3970
353      say   1072.770595   5924.173429   Topic6  -5.1550   0.2750
57        go    903.829889   4099.435591   Topic6  -5.3263   0.4718
141  brother    638.377922   1135.643653   Topic6  -5.6740   1.4077
193     bear    709.750958   3254.468751   Topic6  -5.5680   0.4609

[370 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
2546      1  0.998692   accord
872       4  0.998534      act
832       1  0.999659    actor
1048      3  0.998811  actress
741       2  0.227589      add
...     ...       ...      ...
7         2  0.232342     year
7         3  0.002001     year
7         4  0.005276     year
7         5  0.082966     year
7         6  0.027655     year

[664 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 4, 6, 5, 3])